<a href="https://colab.research.google.com/github/Kibogora/ThriveTalk/blob/main/MentalHealthbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/MLDatasets/Mental_Health_FAQ.csv')
df.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [26]:
df.columns

Index(['Question_ID', 'Questions', 'Answers'], dtype='object')

In [27]:
import pandas as pd
from sentence_transformers import SentenceTransformer
# Assuming your DataFrame is already loaded as 'df'

context_data = []

for i in range(len(df)):
  context = f"Question: {df.iloc[i]['Questions']} Answer: {df.iloc[i]['Answers']}"
  context_data.append(context)
# print(context_data)

# Embed the contexts
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
context_embeddings = embedding_model.encode(context_data)

print(f"Number of contexts: {len(context_data)}")
print(f"Shape of embeddings: {context_embeddings.shape}")

Number of contexts: 98
Shape of embeddings: (98, 384)


In [28]:
len(context_data)

98

In [29]:
context_data[0]

'Question: What does it mean to have a mental illness? Answer: Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical condition due to the brainâ€™s biology.\nS

In [30]:
from google.colab import userdata
groq_key = userdata.get('Chatbot_AIMSRic')

In [31]:
!pip install langchain_groq langchain_huggingface langchain_chroma

In [32]:
## LLM used for RAG
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-70b-versatile",api_key=groq_key)

In [33]:
## Embedding model!
from langchain_huggingface import HuggingFaceEmbeddings
embed_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

In [34]:
# create vector store!
from langchain_chroma import Chroma

vectorstore = Chroma(
    collection_name="medical_dataset_store",
    embedding_function=embed_model,
    persist_directory="./",
)


In [35]:
vectorstore.get().keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'included'])

In [36]:
# add data to vector nstore
vectorstore.add_texts(context_data)

['a706fd0b-516c-40e6-87ea-ac3b39658fe3',
 '652a83c9-3f5f-4335-b934-614791bfc2d9',
 '867ea1a4-0be2-4b7f-a32d-4e3348c576d9',
 '248b131a-3264-45a3-a3dc-d9bfa6a98959',
 '3790df19-494e-4aae-81fc-e47714d5b7be',
 '015ef40a-fc22-4e16-aa64-693d93afba9a',
 '36ec24c9-3496-44aa-a240-4804b7871ee9',
 '3654b2ee-9ca9-4278-ad3c-1ddc4224a620',
 '503a86d3-3e90-41b0-9251-ea752b6bff3c',
 'c6b2fd63-fe35-4308-b60b-ffecf6b6c101',
 'c9f1ee2c-7ee9-4920-8454-e37334eb0e70',
 'dae25a15-8290-47ce-bbf4-c20cbd207bb4',
 '8b17d1e7-6967-4539-91d7-c59e741e7af9',
 'a8efa956-4b0a-4259-800c-0fb50bac0114',
 'f776eda1-c350-4099-a908-8d91d997d375',
 '643366dd-dff2-4cce-9902-9293f56ffe8a',
 '3558eecd-b93e-4be2-8ea4-3e609b216b33',
 '94059ad0-f68e-41bc-9b10-c1b7a09a029c',
 '898ba922-fdb5-4392-81dd-1b50f80cdf95',
 '90eea3fb-fd8a-47a5-907d-412396140605',
 '59bd65cf-5bd1-4fa3-a8af-64cf2cfaa47a',
 '4ba0b05d-f67c-429a-bfcf-d7b2e5cdb59d',
 '40e43791-6896-4d23-8fe1-f7f537f0891f',
 'ebd5b0ab-62fa-4d3f-a63f-b2d85971c1ec',
 '6d1e6a01-804a-

In [37]:
query = "What is the Symptoms anxiety and depression?"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

Question: What are some of the warning signs of mental illness? Answer: Symptoms of mental health disorders vary depending on the type and severity of the condition. The following is a list of general symptoms that may suggest a mental health disorder, particularly when multiple symptoms are expressed at once.
In adults:
Confused thinking
Long-lasting sadness or irritability
Extreme highs and lows in mood
Excessive fear, worrying, or anxiety
Social withdrawal
Dramatic changes in eating or sleeping habits
Strong feelings of anger
Delusions or hallucinations (seeing or hearing things that are not really there)
Increasing inability to cope with daily problems and activities
Thoughts of suicide
Denial of obvious problems
Many unexplained physical problems
Abuse of drugs and/or alcohol
  In older children and pre-teens:
Abuse of drugs and/or alcohol
Inability to cope with daily problems and activities
Changes in sleeping and/or eating habits
Excessive complaints of physical problems
Defying

In [38]:
retriever = vectorstore.as_retriever()

In [39]:
from langchain_core.prompts import PromptTemplate

template = ("""You are a medical expert.
    Use the provided context to answer the question.
    If you don't know the answer, say so. Explain your answer in detail.
    Do not discuss the context in your response; just provide the answer directly.

    Context: {context}

    Question: {question}

    Answer:""")

rag_prompt = PromptTemplate.from_template(template)

In [40]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [41]:
from IPython.display import display, Markdown

response = rag_chain.invoke("What is the symptoms meaning of bipolar?")
Markdown(response)

Bipolar disorder is characterized by episodes of hypomania and depression. The symptoms of hypomania include three or more symptoms, although the specific symptoms are not provided in the given context. The symptoms of depression include five or more symptoms, but again, the specific symptoms are not detailed in the context. 

In general, as a medical expert, I can explain that hypomania symptoms may include elevated mood, increased energy, reduced need for sleep, increased talkativeness, and impulsive behavior, among others. Depression symptoms may include persistent feelings of sadness, loss of interest in activities, changes in appetite or sleep, fatigue, and difficulty concentrating.

It's essential to note that the provided context does not explicitly list the symptoms of hypomania and depression. However, based on general medical knowledge, the symptoms mentioned above are commonly associated with these conditions. If you think you might have bipolar disorder, it's crucial to consult a doctor or mental health professional for an accurate diagnosis and treatment.

In [42]:
from IPython.display import display, Markdown

response = rag_chain.invoke("What do you think women wants?")
Markdown(response)

I don't know the answer to that question. The provided context does not offer any information about what women want, and it's a complex and individualized topic that cannot be generalized. As a medical expert, I can provide information on various health-related topics, but I don't have enough data to make an informed statement about what women want. It's essential to recognize that every individual is unique, and their desires, needs, and preferences can vary greatly.

In [43]:
## Deployment

In [44]:
!pip install gradio

In [45]:
import gradio as gr

def rag_memory_stream(text):
    partial_text = ""
    for new_text in rag_chain.stream(text):
        partial_text += new_text
        yield partial_text


title = "Real-time AI App with Groq API and LangChain to Answer medical questions"
demo = gr.Interface(
    title=title,
    fn=rag_memory_stream,
    inputs="text",
    outputs="text",
    allow_flagging="never",
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3421fa7830e928d41f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
